In [83]:
import pandas as pd
import numpy as np
from math import isnan
from sklearn.metrics import confusion_matrix as confmat
from sklearn.metrics import f1_score as f1
# from sklearn.metrics import plot_confusion_matrix as plot_confmat

# import re
# import string
import random
import importlib

# import nltk
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer

import torch
import preprocess
import dan

import gensim
# load dataset and preprocess
# twt = pd.read_csv('train.csv')
twt = pd.read_csv('train.csv')
twt = twt.set_index('id')
# twt = twt.loc[twt.index < 20000,]
add = pd.read_csv('tweets_0_300.csv')
add = add.set_index('id')
twt.head()

print(twt['target'].value_counts()/len(twt['target']))
print(add['target'].value_counts()/len(add['target']))

# create text+ column which includes location
twt['text+'] = twt.apply(lambda x: x['text'] if pd.isna(x['location']) else  x['location'] + ' ' + x['text'], axis = 1) 

twt['text++'] = twt.apply(lambda x: x['text+'] if pd.isna(x['keyword']) else x['keyword'] + ' ' + x['text+'], axis = 1)
twt.loc[~twt['keyword'].isna(),]

0    0.57034
1    0.42966
Name: target, dtype: float64
0    1.0
Name: target, dtype: float64


,keyword,location,text,target,text+,text++
id,,,,,,
48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,Birmingham @bbcmtd Wholesale Markets ablaze ht...,ablaze Birmingham @bbcmtd Wholesale Markets ab...
49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,Est. September 2012 - Bristol We always try to...,ablaze Est. September 2012 - Bristol We always...
50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,AFRICA #AFRICANBAZE: Breaking news:Nigeria fla...,ablaze AFRICA #AFRICANBAZE: Breaking news:Nige...
52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,"Philadelphia, PA Crying out for more! Set me a...","ablaze Philadelphia, PA Crying out for more! S..."
53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,"London, UK On plus side LOOK AT THE SKY LAST N...","ablaze London, UK On plus side LOOK AT THE SKY..."
...,...,...,...,...,...,...
10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0,@jt_ruff23 @cameronhacker and I wrecked you both,wrecked @jt_ruff23 @cameronhacker and I wrecke...
10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,"Vancouver, Canada Three days off from work and...","wrecked Vancouver, Canada Three days off from ..."
10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,London #FX #forex #trading Cramer: Iger's 3 w...,wrecked London #FX #forex #trading Cramer: Ig...


## 0. Data Preprocessing

### 0.1 Data Cleaning

In [74]:
text = twt['text+'].to_list()
prep_text = [preprocess.processing(i) for i in text]
print('{} tweets lowered, tokenized, alphanumerized, stop-stripped, and stemmed.'.format(len(prep_text)))  
twt['prepped'] = prep_text
twt.tail()

7613 tweets lowered, tokenized, alphanumerized, stop-stripped, and stemmed.


,keyword,location,text,target,text+,prepped
id,,,,,,
10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,Two giant cranes holding a bridge collapse int...,"[two, giant, cranes, holding, a, bridge, colla..."
10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,@aria_ahrary @TheTawniest The out of control w...,"[ariaahrary, thetawniest, the, out, of, contro..."
10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,"[m, utckm, s, of, volcano, hawaii, httptcozdto..."
10872,NaN,NaN,Police investigating after an e-bike collided ...,1,Police investigating after an e-bike collided ...,"[police, investigating, after, an, ebike, coll..."
10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1,The Latest: More Homes Razed by Northern Calif...,"[the, latest, more, homes, razed, by, northern..."


### 0.2 Embedding

In [75]:
# word2vec = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
importlib.reload(preprocess)
twt['vec'] = pd.Series([preprocess.tweet_vec(tweet, word2vec) for tweet in twt['prepped']], index = twt.index)

devtrain_idx = twt.loc[~twt['vec'].isna()].index.tolist()
random.shuffle(devtrain_idx)

train_p = 0.90
train_idx = devtrain_idx[:round(train_p*len(devtrain_idx))]
dev_idx = devtrain_idx[round(train_p*len(devtrain_idx)):]

train = [(vec, targ) for targ, vec in  zip(twt['target'][train_idx], twt['vec'][train_idx])]
dev = [(vec, targ) for targ, vec in  zip(twt['target'][dev_idx], twt['vec'][dev_idx])]

## 1. NN construction

In [76]:
import importlib
importlib.reload(dan)
net = dan.Net(hiddenDim = 2)
net.train(train, dev)

[1,     1] loss: 0.701	Dev FI: 0.629
[1,     2] loss: 0.692	Dev FI: 0.353
[2,     1] loss: 0.683	Dev FI: 0.016
[2,     2] loss: 0.675	Dev FI: 0.004
[3,     1] loss: 0.670	Dev FI: 0.000
[3,     2] loss: 0.662	Dev FI: 0.000
[4,     1] loss: 0.660	Dev FI: 0.004
[4,     2] loss: 0.652	Dev FI: 0.006
[5,     1] loss: 0.652	Dev FI: 0.012
[5,     2] loss: 0.643	Dev FI: 0.018
[6,     1] loss: 0.645	Dev FI: 0.045
[6,     2] loss: 0.635	Dev FI: 0.119
[7,     1] loss: 0.636	Dev FI: 0.232
[7,     2] loss: 0.625	Dev FI: 0.329
[8,     1] loss: 0.625	Dev FI: 0.411
[8,     2] loss: 0.613	Dev FI: 0.507
[9,     1] loss: 0.613	Dev FI: 0.571
[9,     2] loss: 0.602	Dev FI: 0.629
[10,     1] loss: 0.601	Dev FI: 0.656
[10,     2] loss: 0.592	Dev FI: 0.686
[11,     1] loss: 0.591	Dev FI: 0.703
[11,     2] loss: 0.583	Dev FI: 0.709
[12,     1] loss: 0.583	Dev FI: 0.711
[12,     2] loss: 0.575	Dev FI: 0.715
[13,     1] loss: 0.575	Dev FI: 0.716
[13,     2] loss: 0.568	Dev FI: 0.715
[14,     1] loss: 0.568	Dev FI

## Dev Performance and Error Analysis

In [77]:
# dev[0]
ys, y_stars = net.get_eval_data(dev)
# confmat(ys, y_stars)
print('Dev F1: {}'.format(f1(ys, y_stars)))

twt['pred'] = pd.Series(y_stars, index = dev_idx)


# for error analysis
twt[['keyword', 'location', 'text', 'prepped','vec', 'target','pred']].to_csv('error_analysis.csv')

Dev F1: 0.7530536378120021


## Create Test Submissions

In [79]:
# create vector of mean of all word vectors
mean_vec = np.zeros((300,1))
for vec_targ in train:
    mean_vec = np.add(mean_vec, vec_targ[0])
mean_vec = mean_vec/len(train)

# read in test data and preprocess tweets
test = pd.read_csv('test.csv')
text = test['text']
proc_text = [preprocess.processing(i) for i in text]
targets = np.zeros((len(test), 1))
print('{} tweets read from test.csv'.format(test.shape[0]))


# test_data = [(tweet_vec(x, word2vec), y) if for x, y in zip(proc_test, targets)]
test['vec'] = pd.Series([preprocess.tweet_vec(tweet, word2vec) for tweet in proc_text], index = test.index)

print('{} tweets processed in the test set.'.format(test.shape[0]))

test_data = test['vec'].tolist()


_, y_stars = net.get_eval_data(test_data, mode = 'test')

# create columns for submission data
id = test['id'].to_numpy()
target = y_stars

# create df with submission data and write to csv
submission = pd.DataFrame({'id': id, 'target': target})
submission.set_index('id').to_csv('submission.csv')

# also save analysis csv for reviewing decisions
test['prepped'] = proc_text
test['pred'] = target
test[['keyword', 'location', 'text', 'prepped','pred']].to_csv('test_error_analysis.csv')

3263 tweets read from test.csv
3263 tweets processed in the test set.


In [43]:
test[['id', 'pred']].set_index('id').to_csv('submission.csv')

## 2. Naive Bayes with Bag of Words

In [44]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# create raw bow text and labels, shuffle
bow = twt['text'].to_numpy()
labels = twt['target'].to_numpy()

shuffle_idx = [x for x in range(len(bow))]
random.shuffle(shuffle_idx)

bow = bow[shuffle_idx]
labels = labels[shuffle_idx]

# train bag-of-words
count_vect = CountVectorizer()
x_bow = count_vect.fit(bow)

# create train dev split
train_bow = bow[:round(train_p*len(bow))]
dev_bow = bow[round(train_p*len(bow)):]
train_bow = count_vect.transform(train_bow)
dev_bow = count_vect.transform(dev_bow)

train_bow_labels = labels[:round(train_p*len(bow))]
dev_bow_lables = labels[round(train_p*len(bow)):]

print('Created {} bag-of-words vectorizations.'.format(train_bow.shape[0] + dev_bow.shape[0]))
clf = MultinomialNB().fit(train_bow, train_bow_labels)
print('Naive Bayes training complete.')

# get dev performances
ys = clf.predict(dev_bow)

# dev performance
print('Naive Bayes dev performance {}'.format(f1(ys, dev_bow_lables)))

Created 7613 bag-of-words vectorizations.
Naive Bayes training complete.
Naive Bayes dev performance 0.7500000000000001


## Missingness Exploration

We suspect that location data can be useful for predicting disasters. However, we have a sizable proportion of the data for which the location data is missing. In order to address this issue, we needed to first identify whether the data is missing completely at random, missing at random, or not missing at random. For the data to be MCAR, the mechanism of missingness of the data must be independent of all other observed features of the data. These other observed features, in our case, include the text of the tweet. Reviewing the Twitter policy on tweet-level location data, we found that twitter users must opt in to location services to embed location data in there tweets, as well as intentionally include location data on each desired tweet. This policy gives us an insight into some mechanisms of missingness - users forget that they can include location data, or the intentionally elect not to for some tweets. This second mechanism would preclude categorization as missing completely at random. Furthermore, as users of social media, the authors have firsthand experience with the relationship between the location from which a tweet was sent, and the inclusion of location data. Twitter users may want to inform followers that they are tweeting from an impressive or otherwise unusual location - a famous concert venue, or a historic city, or even the site of a terrible natural disaster - a decision surely not made at random. Being Missing Not a Random, our options for mitigating location data missingness are few. We could drop the data altogether, but decided that appending it to the contents of the tweet would be an acceptable approach that prevents information loss.


## Hyperparameter Tuning

### Hidden Layers and Preprocessing

In [64]:
# hidden Layer dimensions
pre_opts = [(1, 0, 0), (1, 1, 0), (1, 0, 1) , (1, 1, 1), (0, 1, 0), (0, 1, 1), (0, 0, 1)]
Fs_ls = []
for column in ['text', 'text+', 'text++']:
    text = twt[column]
    for lower, stopwords, stem in pre_opts:
        prep_text = [preprocess.processing(i, lower, stopwords, stem) for i in text]
        # print('{} tweets lowered, tokenized, alphanumerized, stop-stripped, and stemmed.'.format(len(prep_text)))

        twt['vec'] = pd.Series([preprocess.tweet_vec(tweet, word2vec) for tweet in prep_text], index = twt.index)

        devtrain_idx = twt.loc[~twt['vec'].isna()].index.tolist()
        random.shuffle(devtrain_idx)

        train_p = 0.7
        train_idx = devtrain_idx[:round(train_p*len(devtrain_idx))]
        dev_idx = devtrain_idx[round(train_p*len(devtrain_idx)):]

        train = [(vec, targ) for targ, vec in  zip(twt['target'][train_idx], twt['vec'][train_idx])]
        dev = [(vec, targ) for targ, vec in  zip(twt['target'][dev_idx], twt['vec'][dev_idx])]

        Fs = []
        for hiddenDim in range(2,150,10):
            # print('Executing model with hidden layer dimensionality {}'.format(hiddenDim))
            net = dan.Net(hiddenDim)
            net.train(train, dev, verbose = False)
            ys, y_stars = net.get_eval_da
            ta(dev)
            Fs.append(f1(ys, y_stars))
            print('Finished training')
        Fs_ls.append(Fs)

results = {i : Fs_ls[i] for i in range(len(Fs_ls))}
pd.DataFrame(results).to_csv('hiddenDim_preprocessing.csv')

Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished training
Finished t

In a grid search dev evaluation of the hiddenDim and preprocessing hyperparameters, we found that hiddenDim = 52, lowercasing, and stopword inclusion (1, 1, 0) provided the best results, at F1 = 0.744

### Size and Class Balance of Training Data

In [25]:
samples = [pd.concat([twt, add.sample(n = N)], axis = 0) for N in [0, 2000, 5000, 10000, 15000]]

Fs_ls  = []
FPR_ls = []
for sample in samples:
    text = sample['text']
    prep_text = [preprocess.processing(i) for i in text]
    # print('{} tweets lowered, tokenized, alphanumerized, stop-stripped, and stemmed.'.format(len(prep_text)))

    sample['vec'] = pd.Series([preprocess.tweet_vec(tweet, word2vec) for tweet in prep_text], index = sample.index)

    devtrain_idx = sample.loc[~sample['vec'].isna()].index.tolist()
    random.shuffle(devtrain_idx)

    train_p = 0.7
    train_idx = devtrain_idx[:round(train_p*len(devtrain_idx))]
    dev_idx = devtrain_idx[round(train_p*len(devtrain_idx)):]

    train = [(vec, targ) for targ, vec in  zip(sample['target'][train_idx], sample['vec'][train_idx])]
    dev = [(vec, targ) for targ, vec in  zip(sample['target'][dev_idx], sample['vec'][dev_idx])]

    Fs = []
    FPR = []
    for hiddenDim in range(2,302,25):
        # print('Executing model with hidden layer dimensionality {}'.format(hiddenDim))
        net = dan.Net(hiddenDim)
        net.train(train, dev, verbose = False)
        ys, y_stars = net.get_eval_data(dev)
        Fs.append(f1(ys, y_stars))
        # false positive rate
        FPR.append(np.sum([1 if (y == 0) and (y_star == 1) else 0 for y, y_star in zip(ys, y_stars)])/(len(ys) - np.sum(ys)))
    Fs_ls.append(Fs)
    FPR_ls.append(FPR)

f1_results = {i : Fs_ls[i] for i in range(len(Fs_ls))}
fpr_results = {i : FPR_ls[i] for i in range(len(FPR_ls))}
pd.DataFrame(fpr_results).to_csv('data_dim_ablation_fpr.csv')

Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Training.
Finished Tra

## Learning Rate, Epochs and Optimizer

In [29]:
import importlib
importlib.reload(dan)
Fs_ls  = []
text = twt['text']
prep_text = [preprocess.processing(i) for i in text]
# print('{} tweets lowered, tokenized, alphanumerized, stop-stripped, and stemmed.'.format(len(prep_text)))

twt['vec'] = pd.Series([preprocess.tweet_vec(tweet, word2vec) for tweet in prep_text], index = twt.index)

devtrain_idx = twt.loc[~twt['vec'].isna()].index.tolist()
random.shuffle(devtrain_idx)

train_p = 0.7
train_idx = devtrain_idx[:round(train_p*len(devtrain_idx))]
dev_idx = devtrain_idx[round(train_p*len(devtrain_idx)):]

train = [(vec, targ) for targ, vec in  zip(sample['target'][train_idx], sample['vec'][train_idx])]
dev = [(vec, targ) for targ, vec in  zip(sample['target'][dev_idx], sample['vec'][dev_idx])]

Fs = []
for opter in ['adam', 'sgd']:
    # print('Executing model with hidden layer dimensionality {}'.format(hiddenDim))
    for epochs in [5, 10, 50, 100]:
        Fs = []
        for lr in np.logspace(-4, -1, 7):
            net = dan.Net(hiddenDim)
            net.train(train, dev, verbose = False, opter = opter, lr = lr, epochs = epochs)
            ys, y_stars = net.get_eval_data(dev)
            Fs.append(f1(ys, y_stars))
            # false positive rate
        Fs_ls.append(Fs)

f1_results = {i : Fs_ls[i] for i in range(len(Fs_ls))}
pd.DataFrame(f1_results).to_csv('epochs_lr_opter_ablation.csv')